In [ ]:
%pip install pycryptodome
%pip install filetype

In [1]:
from Crypto.Cipher import AES
import filetype
from Crypto.PublicKey import RSA
from glob import glob
from math import gcd
import os

In [2]:
ficheros = glob('./*.pem')

with open(ficheros[-1], 'r') as f:
        key_encoded = f.read()

pubkey = RSA.importKey(key_encoded)
personal = pubkey.n
public_exponent = pubkey.e
print(personal)
print()
print(public_exponent)

12800032670689292379526711945669482043487644976317237084263536174427345983772924727880182264579537914882316347485871273755269325840083142846267548498275386577325621609154693549802432360724902595749054391305874152266688458518322778001102404549748858924981207582702596762785750401021852347195467284376043073127222606912814391000794852408679675720120427317505218600443973367638315189539502819941309684926841578467131023876206996157169195958796218132479233059116942731820413857791016251857680764505251311773878791754973215607683168941534093112281587782163869950615984559634404885208421648577538375731290884054354814931151

65537


In [3]:
ficheros = glob('./publickeys/*')
posibles = []
for fichero in ficheros:
    with open(fichero, 'r') as f:
        key_encoded = f.read()
        
    pubkey = RSA.importKey(key_encoded)
    others = pubkey.n
    p = gcd(personal, others)
    
    if p != 1:
        print(fichero)
        posibles.append(p)
    

./publickeys/marcel.urpi_pubkeyRSA_RW.pem
./publickeys/adrian.villar_pubkeyRSA_RW.pem


In [4]:
p = posibles[0]
q = posibles[1]

(p*q) == personal

True

In [5]:
inverse = (2**16+1)^-1
phi = (p-1)*(q-1)

private_exponent = mod(inverse, phi)

In [6]:
key = RSA.construct((int(personal), int(public_exponent), int(private_exponent)), consistency_check=False)

f = open('privateKey_Pablo_Vega.pem','wb')
f.write(key.export_key('PEM'))
f.close()

In [7]:
os.system('openssl rsautl -decrypt -in pablo.vega.gallego_RSA_RW.enc -out AESkey.txt -inkey privateKey_Pablo_Vega.pem')

0

In [8]:
with open('AESkey.txt', 'rb') as f:
    AESkey = f.read()

In [9]:
os.system('openssl enc -d -aes-128-cbc -pbkdf2 -kfile AESkey.txt -in pablo.vega.gallego_AES_RW.enc -out desencriptado.txt')

0

In [10]:
with open('desencriptado.txt', 'rb') as f:
    desencriptado = f.read()

In [11]:
filetype.guess(desencriptado)

In [12]:
with open('foto.png', "wb") as binary_file:
        binary_file.write(desencriptado)